In [3]:
import sys
from pathlib import Path
from copy import deepcopy
import torch
import matplotlib.pyplot as plt
import numpy as np

try:
    if not(changed):
        raise Exception()
except:
    sys.path.append(str(Path(".").absolute().parent))
    changed = True

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
from ocpmodels.common.flags import flags
from ocpmodels.common.utils import build_config, resolve, setup_imports, merge_dicts
from ocpmodels.trainers.single_trainer import SingleTrainer

In [5]:
runs_dir = "$SCRATCH/ocp/runs"

def load_checkpoint(job_id):
    path = resolve(runs_dir) / job_id

    checkpoint = path / "checkpoints" / "best_checkpoint.pt"
    setup_imports()
    argv = deepcopy(sys.argv)
    # trainer_args = flags.parser.parse_args()
    trainer_args = flags.parser.parse_args([])
    sys.argv[1:] = argv
    trainer_args.continue_from_dir = str(path)
    config = build_config(trainer_args, [])
    config["logger"] = "dummy"
    config["checkpoint"] = str(checkpoint)
    # config["optim"]["batch_size"] = 1
    # config["optim"]["eval_batch_size"] = 1
    print(config)

    trainer = SingleTrainer(**config)
    
    return trainer, config


In [6]:
job_id = "4082986"
trainer, config = load_checkpoint(job_id)

# job_id2 = "4115585"
# trainer_is2re, config_is2re = load_checkpoint(job_id2)


🅾️  Removing orion config from continue config. Set to None: {}
✅ Loading config from /network/scratch/a/ali.ramlaoui/ocp/runs/4082986/checkpoints/checkpoint-0125004.pt (and loading latest checkpoint)
🏭 Overriding num_workers from 4 to 1 to match the machine's CPUs. Use --no_cpus_to_workers=true to disable this behavior.
{'frame_averaging': '2D', 'fa_method': 'random', 'model': {'name': 'faenet', 'act': 'swish', 'hidden_channels': 256, 'num_filters': 480, 'num_interactions': 7, 'num_gaussians': 136, 'dropout_lin': 0.0, 'dropout_edge': 0.0, 'dropout_lowest_layer': 'output', 'first_trainable_layer': '', 'cutoff': 6.0, 'use_pbc': True, 'regress_forces': 'from_energy', 'tag_hidden_channels': 32, 'pg_hidden_channels': 64, 'phys_embeds': False, 'phys_hidden_channels': 0, 'energy_head': 'weighted-av-final-embeds', 'skip_co': False, 'second_layer_MLP': False, 'complex_mp': True, 'mp_type': 'updownscale_base', 'graph_norm': True, 'force_decoder_type': 'mlp', 'force_decoder_model_config': {'simp

In [ ]:
from time import time

loader = trainer.get_dataloader(trainer.datasets["train"], trainer.samplers["train"])

start_time = time()
for batch in loader:
    pred = trainer.model_forward(batch)
    break
print(time() - start_time)

pred

3.7473878860473633


{'energy': tensor([-5.0742e+00,  1.7985e+01,  5.0195e+00,  2.0770e+00, -4.9448e-01,
          7.3256e+00,  2.8017e+00,  2.1809e+01, -1.6474e+01,  1.6970e+01,
          9.8429e-01, -3.5890e+00,  7.6492e+00, -1.3164e+00,  6.8020e+00,
          7.4567e-01,  1.4545e+01,  7.3455e+00,  2.8984e+00,  1.8467e+00,
          9.4809e+00,  4.1626e+00,  2.9778e-01,  7.7516e+01,  3.4663e+00,
          4.4631e+00,  3.1854e+01,  6.2748e-01,  1.8080e+01,  1.9131e+00,
          4.6608e+00, -9.0478e+00,  2.7776e+00,  7.8159e+00, -4.3001e+00,
          1.4898e+01,  2.6713e+01, -3.8987e+00,  6.2294e+00,  5.1163e+00,
          2.8386e+00,  5.7985e-01,  8.2854e+00, -9.7471e-01,  4.4226e+00,
          5.8316e+00, -9.0830e-01,  3.5892e+00,  1.3096e+00,  2.5293e+00,
          4.6815e+00,  2.4960e+00,  8.1742e+00,  7.7789e+00,  8.3755e+00,
          8.6059e+00,  1.2301e+01,  1.2741e+01,  4.4654e+00,  8.4028e-01,
          9.0567e+00,  4.4115e+00, -6.2747e+00,  6.9291e+00,  1.0186e+01,
         -1.1033e+00,  9.963

In [ ]:
deepcopy(batch[0].fa_pos)

[tensor([[ 2.1544, -4.1699, 28.7975],
         [ 4.3054, -0.3058, 28.4139],
         [-3.2094, -2.4586, 25.5680],
         ...,
         [-1.4642,  0.5973, 20.1997],
         [-2.0712, -1.0874, 20.1999],
         [-2.8320,  0.1384, 17.3392]])]

In [9]:
from ocpmodels.tasks.task import RelaxationTask, S2EFtoIS2RE

# task = RelaxationTask(config)
task = S2EFtoIS2RE(config)
config_name = "faenet-is2re-10k"

task.setup(trainer, config_name)


🔵 Resuming:
  • 

Loading checkpoint from: /network/scratch/a/ali.ramlaoui/ocp/runs/4082986/checkpoints/best_checkpoint.pt onto cuda:0



ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


🏭 Overriding num_workers from 4 to 1 to match the machine's CPUs. Use --no_cpus_to_workers=true to disable this behavior.
Initializing wandb run:  -ft-is2re - 4199128-faenet-s2ef-2M with name:  faenet-s2ef-2M-ft-is2re


wandb: Currently logged in as: aliramlaoui (mila-ocp). Use `wandb login --relogin` to force relogin



--------------------------------------------------------------------------------

Setting max_steps to  782 from max_epochs (20), dataset length (10000), and batch_size (256)

Using max_steps for scheduler -> 782


In [ ]:
trainer.model

OCPDataParallel(
  (module): FAENet(
    (act): Act()
    (distance_expansion): GaussianSmearing()
    (embed_block): EmbeddingBlock(
      (act): Act()
      (phys_emb): PhysEmbedding()
      (period_embedding): Embedding(7, 64)
      (group_embedding): Embedding(19, 64)
      (tag_embedding): Embedding(3, 32)
      (emb): Embedding(85, 96)
      (lin): Linear(in_features=256, out_features=256, bias=True)
      (lin_e1): Linear(in_features=3, out_features=240, bias=True)
      (lin_e12): Linear(in_features=136, out_features=240, bias=True)
    )
    (interaction_blocks): ModuleList(
      (0-6): 7 x InteractionBlock()
    )
    (output_block): OutputBlock(
      (act): Act()
      (lin1): Linear(in_features=256, out_features=128, bias=True)
      (lin2): Linear(in_features=128, out_features=1, bias=True)
      (w_lin): Linear(in_features=256, out_features=1, bias=True)
    )
  )
)

In [7]:
task.run()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


🏭 Overriding num_workers from 4 to 1 to match the machine's CPUs. Use --no_cpus_to_workers=true to disable this behavior.
Initializing wandb run:  test-is2re-ft - 4117072-faenet-s2ef-2M with name:  faenet-s2ef-2M-ft-is2re-test


wandb: Currently logged in as: aliramlaoui (mila-ocp). Use `wandb login --relogin` to force relogin



--------------------------------------------------------------------------------

Setting max_steps to  1042 from max_epochs (20), dataset length (10000), and batch_size (192)

Using max_steps for scheduler -> 1042

--- 🔄 Beginning of Training @ 2024-02-07 08:48:53 ---

Staring from epoch 0 and step 0
Will train for 20 epochs of 53 steps each
Logging  train metrics every 100 steps
Printing train metrics every 10417 steps
Evaluating every 53 steps


Saving checkpoint to /network/scratch/a/ali.ramlaoui/ocp/runs/4117072/checkpoints/checkpoint-0000053.pt

  > energy_mae: 0.7609
  > energy_mse: 1.2446
  > energy_within_threshold: 0.0218
  > total_loss: 0.2635
  > energy_loss: 0.2635
  > epoch: 1.0000
  > val_id_time: 135.1091
  > val_forward_time_mean: 0.0014
  > val_forward_time_std: 0.0001

Saving checkpoint to /network/scratch/a/ali.ramlaoui/ocp/runs/4117072/checkpoints/best_checkpoint.pt

Train metrics at step 53:
  > energy_mae: 8.55e-01
  > energy_mse: 1.09e+00
  > energy_within_thre

                                 Final results                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃ Metric / Split          ┃  val_id ┃ val_ood_cat ┃ val_ood_ads ┃ val_ood_both ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩
│ energy_mae              │ 0.70655 │     0.68974 │     0.77260 │      0.73021 │
│ energy_mse              │ 1.14881 │     1.08249 │     1.25183 │      1.06974 │
│ energy_within_threshold │ 0.02662 │     0.03040 │     0.02075 │      0.02145 │
│ energy_loss             │ 0.24469 │     0.23753 │     0.26720 │      0.25289 │
│ total_loss              │ 0.24469 │     0.23753 │     0.26720 │      0.25289 │
└─────────────────────────┴─────────┴─────────────┴─────────────┴──────────────┘



• Trainer objective set to: 0.7247742245031721



OutOfMemoryError: CUDA out of memory. Tried to allocate 166.00 MiB (GPU 0; 44.48 GiB total capacity; 41.56 GiB already allocated; 33.88 MiB free; 44.00 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF